In [1]:
pip install flasgger

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install demoji

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import pandas as pd
from flask import Flask, Response, jsonify

In [ ]:
app = Flask(__name__)

from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from
import sqlite3
import demoji

# Membuat database SQLite
conn = sqlite3.connect('data.db', check_same_thread=False)
cursor = conn.cursor()

# Membuat tabel untuk menyimpan data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS processed_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        original_text TEXT,
        processed_text TEXT
    )
''')
conn.commit()

app.json_encoder = LazyJSONEncoder
swagger_template = {
    "info": {
        "title":  "API Documentation for Data Processing and Modeling",
        "version": "1.0.0",
        "description": "Dokumentasi API untuk Data Processing dan Modeling"
    },
    "host": "127.0.0.1:5000"
}

swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui":True,
    "specs_route": "/docs/"
}

swagger = Swagger(app, template=swagger_template, config=swagger_config)
#/Users/kok10/.conda/envs/latihan_binar2/docs/
@swag_from("/Users/kok10/.conda/envs/latihan_binar2/docs/hello_world.yml", methods=['GET'])
@app.route('/hello', methods=['GET'])
def hello_word():
    json_response = {
        'status_code': 200,
        'description': "Menyapa Hello Word",
        'data': "Hello Word",
    }

    response_data = jsonify(json_response)
    return response_data    
    

@swag_from("/Users/kok10/.conda/envs/latihan_binar2/docs/text.yml", methods=['GET'])
@app.route('/text', methods=['GET'])
def text():
    json_response = {
        'status_code': 200,
        'description': "text : Original Text ",
        'data': "Hello, Apa kabar Semua",
    }
    
    response_data = jsonify(json_response)
    return response_data

    
@swag_from("/Users/kok10/.conda/envs/latihan_binar2/docs/text_clean.yml", methods=['GET'])
@app.route('/text-clean', methods=['GET'])
def text_clean():
    json_response = {
        'status_code': 200,
        'description': "text-clean : Original Text",
        'data': re.sub(r'[^a-zA-Z0-9]','',"Hello, Apa kabar semua?@#"),
    }
    
    response_data = jsonify(json_response)
    return response_data


@swag_from("/Users/kok10/.conda/envs/latihan_binar2/docs/text_processing.yml", methods=['POST'])
@app.route('/text-processing', methods=['POST'])
def text_processing():
    
    text = request.form.get('text')
    
    json_response = {
        'status_code': 200,
        'description': "Text yang telah diproses",
        'data': re.sub(r'[^a-zA-Z0-9]','', text),
    }
    
    response_data = jsonify(json_response)
    return response_data


@swag_from("/Users/kok10/.conda/envs/latihan_binar2/docs/text_processing.yml", methods=['POST'])
@app.route('/text-processing-save', methods=['POST'])
def text_processing_save():
    
    text = request.form.get('text')
#     processed_text = re.sub(r'[^a-zA-Z0-9]','', text)
    processed_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
   # Menyimpan data ke dalam database
    cursor.execute('''
        INSERT INTO processed_data (original_text, processed_text)
        VALUES (?, ?)
    ''', (text, processed_text))
    conn.commit()
    
    json_response = {
        'status_code': 200,
        'description': "Text yang telah diproses",
        'data': processed_text,
    }
    
    response_data = jsonify(json_response)
    return response_data

    conn.close()


@swag_from("/Users/kok10/.conda/envs/latihan_binar2/docs/text_processing_file.yml", methods=['POST'])
@app.route('/text-processing-file', methods=['POST'])
def text_processing_file():
    
    # upload file
    file = request.files.getlist('file')[0]
    
    # import file csv ke pandas
    df = pd.read_csv(file, encoding='latin1')
    
# Check if the 'text' column exists in the DataFrame
    if 'Tweet' not in df.columns:
        return jsonify({'error': 'Column "Tweet" not found in the CSV file'}), 400
        
        # Ambil teks yang akan diproses dalam format list
    texts = df['Tweet'].tolist()
    
    # lakukan cleansing pada text
    cleaned_text = []
    for Tweet in texts:
        cleaned_text.append(re.sub(r'[^a-zA-Z0-9\s]', '', Tweet))
    
    json_response = {
        'status_code': 200,
        'description': "Text yang telah diproses",
        'data': cleaned_text,
    }
    
    response_data = jsonify(json_response)
    return response_data

@swag_from("/Users/kok10/.conda/envs/latihan_binar2/docs/text_processing_file.yml", methods=['POST'])
@app.route('/text-processing-file-save', methods=['POST'])
def text_processing_file_save():
    
    def cleansing_text(text):
        # Menghapus URL
        text = re.sub(r'http\S+', '', text)    
        # Mengubah ke huruf kecil (lowercase)
        text = text.lower()
        # Menghapus '/na'
        text = text.replace('/na', '')
        # Menghapus emoji
        text = demoji.replace(text, '')
        # Menghapus link
        text = re.sub(r'www\.[^ ]+', '', text)
        # Menghapus karakter khusus (hanya mempertahankan huruf dan angka)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        return text
    
    def remove_words_from_sentence(sentence, words):
        # Menghapus kata yang terdapat dalam list words dari kalimat
        for word in words:
            sentence = re.sub(rf'\b{re.escape(word)}\b', '', sentence)
            # Menghapus kata yang berulang
            sentence = re.sub(r'\b(\w+)\b\s+\b\1\b', r'\1', sentence)
            sentence = sentence.replace('xf0', '')   
            sentence = sentence.replace('x9f', '')    
            sentence = sentence.replace('x8f', '') 
            sentence = sentence.replace('xe2', '')     
            sentence = sentence.replace('x80', '')   
            sentence = sentence.replace('x9c', '')  
            sentence = sentence.replace('x84', '')   
            sentence = sentence.replace('xa3', '')
            sentence = sentence.replace('x98', '')
            sentence = sentence.replace('x86', '')   
            sentence = sentence.replace('xc2', '')
            sentence = sentence.replace('xb2', '')
            sentence = sentence.replace('x9d', '')
            sentence = sentence.replace('xa4', '')
            sentence = sentence.replace('xa2', '')
            sentence = sentence.replace('x99', '')
            sentence = sentence.replace('xab', '')
            sentence = sentence.replace('xaa', '')
            sentence = sentence.replace('xe1', '')
            sentence = sentence.replace('xbd', '')
            sentence = sentence.replace('xcf', '')
            sentence = sentence.replace('xaa', '')
            sentence = sentence.replace('xce', '')
            sentence = sentence.replace('xb5', '')
            sentence = sentence.replace('xcf', '')
            sentence = sentence.replace('x81', '')
            sentence = sentence.replace('xce', '')
            sentence = sentence.replace('xb7', '')
            sentence = sentence.replace('xb1', '')
            sentence = sentence.replace('xaf', '')
            sentence = sentence.replace('xc4', '')
            sentence = sentence.replace('x93', '')
            sentence = sentence.replace('x82', '')
        return sentence

    # Nama tabel dalam database yang berisi kamus alay-baku
    table_name = 'kamusalay'

    # Nama kolom dalam tabel kamusalay yang berisi kata alay
    alay_column = 'kata_alay'

    # Nama kolom dalam tabel kamusalay yang berisi kata baku
    baku_column = 'kata_baku' 
    
    # Nama tabel dalam database yang berisi kamus abusive
    table_name1 = 'tabel_abusive'

    # Nama kolom dalam tabel abusive yang berisi abusive
    abusive_column = 'ABUSIVE'
    
    # upload file
    file = request.files.getlist('file')[0]
    
    # Membaca kamus alay-baku dari database ke dalam DataFrame
    kamus_df = pd.read_sql(f'SELECT {alay_column}, {baku_column} FROM {table_name}', conn)

    # Membaca abusive dari database ke dalam DataFrame
    kamus_df1 = pd.read_sql(f'SELECT {abusive_column} FROM {table_name1}', conn)

    # Membaca data dari file CSV ke dalam DataFrame
    df = pd.read_csv(file, encoding='latin1')

    # Mengecek apakah kolom 'Tweet' ada dalam DataFrame
    if 'Tweet' not in df.columns:
        raise ValueError('Column "Tweet" not found in the CSV file')

    # Ambil teks yang akan diproses dalam format list
    texts = df['Tweet'].tolist()

    # Lakukan cleansing pada text
    cleaned_text = [cleansing_text(Tweet) for Tweet in texts]

    # Mengganti kata alay dengan kata baku dalam kolom 'Tweet'
    for index, row in kamus_df.iterrows():
        alay_word = row[alay_column]
        baku_word = row[baku_column]
        cleaned_text = [re.sub(rf'\b{re.escape(alay_word)}\b', baku_word, Tweet) for Tweet in cleaned_text]

    # Menghilangkan kata yang ada dalam tabel abusive
    for index, row in kamus_df1.iterrows():
        abusive_word = row[abusive_column]
        cleaned_text = [remove_words_from_sentence(text, [abusive_word]) for text in cleaned_text]    

    # Menyimpan hasil cleansing ke dalam tabel baru dalam database
    df['Tweet'] = cleaned_text  # Memperbarui kolom 'Tweet' dengan teks yang sudah di-cleansing
    df.to_sql('cleansed_data', conn, index=False, if_exists='replace')
    
    # Query untuk mendapatkan data dari tabel 'cleansed_data'
    cursor = conn.execute("SELECT * FROM cleansed_data")

    # Mengambil semua hasil query
    data = cursor.fetchall()

    # Menutup koneksi ke database
    conn.close()

    # Mengubah data ke dalam format JSON
    json_data = []
    for row in data:
        json_data.append( row[0],  # Sesuaikan dengan kolom-kolom yang Anda miliki
                        )

    json_response = {
        'status_code': 200,
        'description': "Text yang telah diproses",
        'data': json_data,
    }
    response_data = jsonify(json_response)
    return response_data

 

if __name__ == '__main__' :
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\ewibowo\AppData\Local\Temp\ipykernel_30800\1064097790.py:23: DeprecationWarning: 'app.json_encoder' is deprecated and will be removed in Flask 2.3. Customize 'app.json_provider_class' or 'app.json' instead.
  app.json_encoder = LazyJSONEncoder
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
import sqlite3

conn = sqlite3.connect('data.db')
print("Opened database sucessfully")
print()

print("list data cleansing")
cursor=conn.execute("select * from processed_data;")

for row in cursor:
    print(row)